In [16]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
from sklearn.preprocessing import StandardScaler
data = pd.read_csv('final_merged_data.csv')

# Lagged features
for feature in ['Price Growth Rate', 'Revenue Growth', 'P/E Ratio Annual']:
    data[f'Lag1_{feature}'] = data.groupby('TICKER')[feature].shift(1)

# Log transformations for skewed variables
for feature in ['P/E Ratio Annual', 'Mkt Cap']:
    data[f'Log_{feature}'] = np.log(data[feature] + 1e-6)
    
print(data.columns)

Index(['Company Name_x', 'TICKER', 'Quarter', 'Price', 'Total Ret',
       'Price Ret Ex-Dividend', 'Weighted Mkt Return', 'Volume', 'Dividend',
       'Price Low', 'Price High', 'Shares Outstanding', 'Excess Return',
       'Mkt Cap', 'Price Growth Rate', 'Quart Rev', 'Quart NI', 'Equity',
       'Liabilities', 'Revenue', 'Industry Code', 'Asset', 'Current Asset',
       'EPS', 'Rating', 'Revenue Growth', 'Earnings Growth Rate',
       'P/E Ratio Annual', 'P/E Ratio Quarter', 'Mean Recommendation',
       'Median Recommendation', 'Recommendation StdDev',
       'Number of Recommendations', 'Number of Upgrades',
       'Number of Downgrades', 'Buy Percentage', 'Sell Percentage',
       'Hold Percentage', 'Total Ret Avg +1D', 'Total Ret Avg +5D',
       'Total Ret Avg +10D', 'Total Ret Avg +20D', 'Excess Return Avg +1D',
       'Excess Return Avg +5D', 'Excess Return Avg +10D',
       'Excess Return Avg +20D', 'Lag1_Price Growth Rate',
       'Lag1_Revenue Growth', 'Lag1_P/E Ratio Annua

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [18]:
# SICCD to Industry Mapping (Short Names)
siccd_mapping = {
    range(100, 1000): "Agriculture",
    range(1000, 1500): "Mining",
    range(1500, 1800): "Construction",
    range(2000, 4000): "Manufacturing",
    range(4000, 5000): "Transport",
    range(5000, 6000): "Retail",
    range(6000, 6800): "Finance",
    range(7000, 9000): "Services",
    range(9100, 10000): "Public",
}

detailed_siccd_mapping = {
    range(100, 1000): "Agriculture, Forestry, and Fishing",
    range(1000, 1500): "Mining and Quarrying",
    range(1500, 1800): "Construction",
    range(2000, 2400): "Food and Tobacco Manufacturing",
    range(2400, 2700): "Paper and Printing Manufacturing",
    range(2700, 2800): "Chemical and Pharmaceutical Manufacturing",
    range(2800, 2900): "Plastics and Rubber Products Manufacturing",
    range(3000, 3400): "Metals and Machinery Manufacturing",
    range(3400, 3600): "Electronics Manufacturing",
    range(3600, 3800): "Technology and Semiconductor Manufacturing",
    range(3800, 4000): "Miscellaneous Manufacturing",
    range(4000, 4500): "Transportation and Logistics",
    range(4500, 5000): "Communication Services and Utilities",
    range(5000, 5200): "Wholesale Trade",
    range(5200, 5600): "Retail Trade",
    range(5600, 6000): "Consumer Services",
    range(6000, 6200): "Banks and Credit Institutions",
    range(6200, 6400): "Insurance Companies",
    range(6400, 6800): "Real Estate and Investment Services",
    range(7000, 7300): "Hotels and Entertainment Services",
    range(7300, 7900): "Professional and Business Services",
    range(7900, 8000): "Healthcare and Social Services",
    range(8000, 8900): "Educational and Research Services",
    range(8900, 9000): "Other Services",
    range(9100, 9700): "Public Administration and Government",
    range(9700, 10000): "International Organizations"
}

def map_sic_to_industry(sic_code):
    for key, value in detailed_siccd_mapping.items():
        if sic_code in key:
            return value
    return "Others"
data['Real Industry'] = data['Industry Code'].apply(map_sic_to_industry)

In [ ]:
selected_features = [
    'Weighted Mkt Return',
    'Price Growth Rate',
    'Revenue Growth',
    'Earnings Growth Rate',
    'Mkt Cap',
    'Volume'
]


results_table = []

# Iterate through industries
for industry, group in data.groupby('Real Industry'):
    # Select features and target variable
    X = group[selected_features]
    y = group['Total Ret Avg +5D']

    # Add a constant term
    X = sm.add_constant(X)

    # Clean the data by removing inf, -inf, and NaN values
    X = X.replace([np.inf, -np.inf], np.nan).dropna()
    y = y[X.index]

    # Check for remaining missing values in the target variable
    if y.isna().sum() > 0:
        valid_indices = y.dropna().index
        X = X.loc[valid_indices]
        y = y.loc[valid_indices]

    # Standardize features for numerical stability
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

    # Fit the regression model
    model = sm.OLS(y, X_scaled).fit()

    # Extract beta coefficients and R^2
    betas = model.params.to_dict()  # Get coefficients as a dictionary
    r_squared = model.rsquared  # R-squared value

    # Append results for the industry
    results_table.append({
        'Industry': industry,
        'R^2': r_squared,
        **{f"Beta_{key}": value for key, value in betas.items()}
    })

# Create a DataFrame for the results
results_df = pd.DataFrame(results_table)


# Show the results table
results_df

,Industry,R^2,Beta_const,Beta_Weighted Mkt Return,Beta_Lag1_Price Growth Rate,Beta_Lag1_Revenue Growth,Beta_Earnings Growth Rate,Beta_Lag1_P/E Ratio Annual
0,Banks and Credit Institutions,0.294093,0.0,0.012019,0.014764,0.003692,0.006573,-0.001916
1,Communication Services and Utilities,0.070433,0.0,0.010332,0.008141,-0.001477,0.003816,0.001413
2,Construction,0.153166,0.0,0.015259,0.003208,0.000498,0.009335,-0.012585
3,Consumer Services,0.038094,0.0,0.007109,-0.001622,-0.001106,-0.000640,0.001045
4,Educational and Research Services,0.082984,0.0,0.010882,0.006538,-0.001070,-0.001381,0.002065
5,Electronics Manufacturing,0.208015,0.0,0.012819,0.011559,0.006244,0.010338,-0.009550
6,Food and Tobacco Manufacturing,0.206308,0.0,0.006322,0.019863,-0.001114,-0.007909,0.010126
7,Healthcare and Social Services,0.136963,0.0,0.011741,0.007649,0.000195,0.000311,-0.001655
8,Hotels and Entertainment Services,0.154599,0.0,0.008313,0.008190,-0.002016,0.003488,-0.003367
9,Insurance Companies,0.185025,0.0,0.008733,0.012431,-0.000381,-0.000162,-0.000095


In [23]:
# Initialize a dictionary to store R^2 values
r_squared_results = {}

data['Next Quarter Ret'] = data.groupby('TICKER')['Total Ret'].shift(-1)

# List of dependent variables to evaluate
dependent_vars = {
    "1D_Return": "Total Ret Avg +1D",
    "5D_Return": "Total Ret Avg +5D",
    "10D_Return": "Total Ret Avg +10D",
    "Next_Quarter_Return": "Next Quarter Ret"
}

# Iterate through each dependent variable and compute R^2
for label, y_var in dependent_vars.items():
    # Define the dependent variable
    y = data[y_var]

    # Ensure X and y align after removing missing or invalid values
    X = sm.add_constant(data[selected_features])
    X = X.replace([np.inf, -np.inf], np.nan).dropna()
    y = y[X.index]

    # Check for missing values in y
    if y.isna().sum() > 0:
        valid_indices = y.dropna().index
        X = X.loc[valid_indices]
        y = y.loc[valid_indices]

    # Standardize the features
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

    # Fit the regression model
    model = sm.OLS(y, X_scaled).fit()

    # Store the R^2 value for the current dependent variable
    r_squared_results[label] = model.rsquared

# Convert R^2 results to a DataFrame for better display
r_squared_df = pd.DataFrame(list(r_squared_results.items()), columns=["Metric", "R^2"])

# Output the R^2 table
r_squared_df


,Metric,R^2
0,1D_Return,0.069870
1,5D_Return,0.091834
2,10D_Return,0.241281
3,Next_Quarter_Return,0.001909
